# **Single Fingerprint Deep Classification**
### **JCE - Software Engineering Final Project** 
##### ***By Kobi Amsellem & Zohar Kedem***

##### In this study we want to discovare if Deep Convolutional Neural Network can classify single fingerprint image to find the owner: 
##### **1. Gender** - 2 classes (male/female).
##### **2. Finger name** - 10 classes (right-thumb, ..., right-thumb, ...)
##### **3. Fingerprint type** - 5 classes (left loop, whirl, right loop, tented arch, arch)
##### **4. Same Person** - 2 classes (Same, Different) *(whether or not two fingerprints belong to the same personperson)* 


In [1]:
import os
import copy
import numpy as np
import datasets as ds
import tensorflow as tf
import FPMLmodule.utils as utils
import FPMLmodule.backbones as bb
import FPMLmodule.classifiers as cf
from tensorflow.keras.optimizers import Adam, Nadam, RMSprop
from FPMLmodule.fpml import FPML 
from pathlib import Path

print('TensorFlow Version:', tf.__version__)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
dbStudyOutPath ='./out/{}/'

weightsRN50 = "./weights/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
weightsMNV2 = "./weights/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5"
weightsENB2 = "./weights/efficientnetb2_notop.h5"
weightsINCEPTIONV3 = "./weights/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
weightsXCEPTION = "./weights/xception_weights_tf_dim_ordering_tf_kernels_notop.h5"

TensorFlow Version: 2.8.0


#### **Model Search** 
##### Main method to search after the best model

in this section we define the ````researchBestModel````, a method to search and compare between several different models and hyper parameters and choose the best one for specific dataset.

##### **Stages:**
##### **1.** Backbone Transfer Learning mode comparison - (a) Pretraind wheights and Untrainable, (b) Pretraind wheights and Trainable, (c) Initialize wheight and Trainable.
##### **2.** Backbone comparison. 
##### **3.** Classifier (perceptron) comparison. 
##### **4.** Loss function comparison. 
##### **5.** Learning rates comparison. 
##### **6.** Optimizers comparison. 
##### **7.** Train best configuration for extra epochs on train+validation datasets

In [2]:
def researchBestModel(configureDS, datasets, epochsSearch, epochsBest, path, optimizers, 
                      learningRates, defaultHypers, losses, classifiers, defaultClassifier, 
                      defaultBackbone, backbonesForSearch, verbose=1, top=1):
    trainDs, testDs, valDs = datasets
    imgDim = configureDS.inputDim
    nbClasses = len(configureDS.classNames)
    outFilePath = path + 'ablationHistory.csv'
    topModelsPath = path+'FinalModels/'
    
    print('Ablation for', configureDS.name)
    
    
    baseStudy = {
            "architecture": {
                "backbone": defaultBackbone['backbone'](imgDim, weights=defaultBackbone['weights'], trainable=False),
                "classifier": defaultClassifier(nbClasses, "softmax"),
                "inputLayer": None, 
                "inputDim": imgDim
            },
            "hyperparameters": defaultHypers
        }
    
    
    
    histories = {}
        # **trainingModeHistories, 
        # **bestBackboneHistories, 
        # **classifierHistories, 
        # **lossesHistories,
        # **learningRatesHistories,
        # **optimizersHistories
   
    
    # --------------------------------------------------
    # ---- Stage 1 - Compare Transfer Learning Mode ----
    # --------------------------------------------------
    
    trainingModeStudy = baseStudy
    trainingModeStudy['architecture']['backbone'] = [
                    defaultBackbone['backbone'](imgDim, weights=defaultBackbone['weights'], trainable=False, name='PT&Untrainable'),
                    defaultBackbone['backbone'](imgDim, weights=defaultBackbone['weights'], trainable=True, name='PT&Trainable'),
                    defaultBackbone['backbone'](imgDim, weights=None, trainable=True, name='IN&Trainable')
                    ]    
    trainingModeHistories = utils.researchStudies(trainDs, valDs, {'DNN-TransferLearningMode' : trainingModeStudy}, epochsSearch, verbose)
    histories = {**histories, **trainingModeHistories}
    bestTrainMode = utils.getBestStudyFromHistories(trainingModeHistories)
    trainable, pretrained = 'Trainable' in bestTrainMode, 'PT' in bestTrainMode
    utils.displayStudiesProgress(trainingModeHistories, path, 'Comparison Transfer Learning Mode')
    utils.saveStudyHistory({'DNN-TransferLearningMode' : trainingModeStudy}, trainingModeHistories, outFilePath)
    print("Backbone Transfer Learning Mode Selected - Pretrained:", pretrained, 'Trainable:', trainable)
    
    
    # ------------------------------------------
    # ---- Stage 2 - Compare DNN Backbones -----
    # ------------------------------------------
    backboneConfigs = [backbone['backbone'](imgDim, weights=backbone['weights'], trainable=trainable) for backbone in backbonesForSearch]
    bestBackboneStudy = copy.deepcopy(trainingModeHistories[bestTrainMode]['config'])
    bestBackboneStudy["architecture"]["backbone"] = backboneConfigs
    bestBackboneHistories = utils.researchStudies(trainDs, valDs, {'Backbone': bestBackboneStudy}, epochsSearch, verbose)
    histories = {**histories, **bestBackboneHistories}
    bestBackboneHistories['Backbone'+"_"+defaultBackbone['backbone'].__name__] = trainingModeHistories[bestTrainMode]
    bestBackbone = utils.getBestStudyFromHistories(bestBackboneHistories)
    utils.displayStudiesProgress(bestBackboneHistories, path, 'Comparison DNN Backbone')
    utils.saveStudyHistory({'BestBackbone': bestBackboneStudy}, bestBackboneHistories, outFilePath)
    print("Best DNN Backbone:", bestBackboneHistories[bestBackbone]['config']["architecture"]["backbone"].name)


    # --------------------------------------------
    # ---- Stage 3 - Compare FC Classifiers ----
    # --------------------------------------------
    classifierStudy = copy.deepcopy(bestBackboneHistories[bestBackbone]['config'])
    classifierStudy['architecture']["classifier"] = [classifier(nbClasses, "softmax") for classifier in classifiers]
    classifierHistories = utils.researchStudies(trainDs, valDs, {'Classifiers': classifierStudy}, epochsSearch, verbose)
    histories = {**histories, **classifierHistories}
    classifierHistories['Classifier'+"_"+str(defaultClassifier.__name__)] = bestBackboneHistories[bestBackbone]
    bestClassifier = utils.getBestStudyFromHistories(classifierHistories)
    utils.displayStudiesProgress(classifierHistories, path, 'Comparison Classifiers')
    utils.saveStudyHistory({'bestClassifier': classifierStudy}, classifierHistories, outFilePath)
    print("Best Classifier:", str(classifierHistories[bestClassifier]['config']['architecture']["classifier"]))
    
    # ------------------------------------------
    # ---- Stage 4 - Compare Loss Functions ----
    # ------------------------------------------
    lossStudy = copy.deepcopy(classifierHistories[bestClassifier]['config'])
    lossStudy['hyperparameters']["loss"] = losses
    lossesHistories = utils.researchStudies(trainDs, valDs, {'LossFunction': lossStudy}, epochsSearch, verbose)
    histories = {**histories, **lossesHistories}
    lossesHistories['LossFunction'+"_"+str(defaultHypers["loss"])] = classifierHistories[bestClassifier]
    bestLoss = utils.getBestStudyFromHistories(lossesHistories)
    utils.displayStudiesProgress(lossesHistories, path, 'Comparison Loss Functions')
    utils.saveStudyHistory({'BestLoss': lossStudy}, lossesHistories, outFilePath)
    print("Best Loss Function:", str(lossesHistories[bestLoss]['config']['hyperparameters']["loss"]))
    
    
    # ------------------------------------------
    # ---- Stage 5 - Compare Learning Rates ----
    # ------------------------------------------
    learningRatesStudy = copy.deepcopy(lossesHistories[bestLoss]['config'])
    learningRatesStudy['hyperparameters']['learningRate'] = learningRates
    learningRatesHistories = utils.researchStudies(trainDs, valDs, {'LearningRate': learningRatesStudy}, epochsSearch, verbose)
    histories = {**histories, **learningRatesHistories}
    learningRatesHistories['LearningRate'+"_"+str(defaultHypers["learningRate"])] = lossesHistories[bestLoss]
    bestLearningRate = utils.getBestStudyFromHistories(learningRatesHistories)
    utils.displayStudiesProgress(learningRatesHistories, path, 'Comparison Learning Rate')
    utils.saveStudyHistory({'BestLearningRate': learningRatesStudy}, learningRatesHistories, outFilePath)
    print("Best Learning Rate:", str(learningRatesHistories[bestLearningRate]['config']["hyperparameters"]["learningRate"]))


    # --------------------------------------
    # ---- Stage 6 - Compare Optimizers ----
    # --------------------------------------
    optimizersStudy = copy.deepcopy(learningRatesHistories[bestLearningRate]['config'])
    optimizersStudy["hyperparameters"]["optimizer"] = optimizers
    optimizersHistories = utils.researchStudies(trainDs, valDs, {'Optimizer' : optimizersStudy}, epochsSearch, verbose)
    histories = {**histories, **optimizersHistories}
    optimizersHistories['Optimizer'+"_"+defaultHypers['optimizer'].__name__] = learningRatesHistories[bestLearningRate]
    bestOptimizier = utils.getBestStudyFromHistories(optimizersHistories)
    utils.displayStudiesProgress(optimizersHistories, path, 'Comparison Optimizer')
    utils.saveStudyHistory({'BestOptimizier': optimizersStudy}, optimizersHistories, outFilePath)
    print("Best Optimizier:", optimizersHistories[bestOptimizier]['config']["hyperparameters"]["optimizer"].__name__)
    
    
    
    
    # ----------------------------------------------------------------
    # ---- Stage 7 - Train top K Best models on train+validation -----
    # ----------------------------------------------------------------
    
   
    
    
    Path(topModelsPath).mkdir(parents=True, exist_ok=True)
    topKmodelsNames = utils.getBestStudyFromHistories(histories, top)
    topModels = {x: histories.get(x, None) for x in topKmodelsNames}
    topModelsTrainedHistories = {}
    for modelName, model in topModels.items():
        history = model['model'].fit(trainDs.concatenate(valDs), testDs, epochsBest, verbose=1)
        model['model'].save(topModelsPath+modelName)
        topModelsTrainedHistories[modelName] = {
            "history":history,
            "model": model['model'],
            "config": model['config']
        }
    
    # utils.saveStudyHistory({'Best Models': None}, topModelsTrainedHistories, topModelsPath+'topHistories.csv', True)
    utils.displayStudiesProgress(topModelsTrainedHistories, topModelsPath)
    return topModelsTrainedHistories
    


In [3]:
def researchStudy(study, path=None):
    datasets = study['datasets']
    studyParams = study['studyHyperParameters']
    preparedDatasets = utils.prepareDatasetsForStudy(datasets, path)
    bestModels = {}
    for dsName, dsObj in preparedDatasets.items():
        bestModels[dsName] = researchBestModel(**dsObj, **studyParams)
    return bestModels


##### **Datasets Configuration**

In [4]:
datasetsConfig = {
    'batchSize': 32, 
    'parallelTune': tf.data.AUTOTUNE, 
    'split': [0.7, 0.15, 0.15], 
    'inputDim': (224, 224, 3), 
    'seed': 9, 
    'shuffle': True
    }

#### **2. Gender** 
##### **Datasets:**
* **SOCOFing**
* **NIST Special Database 4**

##### **Hyperparameters**
* Optimizers: *Adam, Nadam, RMSprop*
* Loss Functions: *binary-crossentropy, mean-squared-error, hinge*
* Learning Rates: *0.1, 0.01, 0.001, 0.0001, 0.00001*
* Backbones: *MobileNetV2, ResNet50, EfficientNetB2, InceptionV3, Xception*

In [17]:

genderStudy = {
    'datasets' : [
        ds.SOCOFingGender(**datasetsConfig, sampling=ds.SOCOFingGender.UNDER_SAMPLING), 
        ds.NISTSDB4Gender(**datasetsConfig), 
    ],
    'studyHyperParameters' : {
        'defaultBackbone': {'backbone' : bb.MobileNetV2, 'weights' : weightsMNV2},
        'defaultClassifier': cf.DefaultClassifier,
        'defaultHypers': {
            "optimizer": Adam,
            "learningRate": 0.001,
            "loss": 'binary_crossentropy',
            "metrics": 'accuracy'
        },
        'losses': ['mean_squared_error', 'hinge'],
        'learningRates': [0.1, 0.01, 0.0001, 0.00001],
        'optimizers': [Nadam, RMSprop],
        'classifiers': [cf.AlexNetClassifier, cf.MobileNetClassifier, cf.ResNetClassifier],
        'backbonesForSearch': [
            {'backbone' : bb.ResNet50, 'weights' : weightsRN50},
            {'backbone' : bb.EfficientNetB2, 'weights' : weightsENB2},
            {'backbone' : bb.InceptionV3, 'weights' : weightsINCEPTIONV3},
            {'backbone' : bb.Xception, 'weights' : weightsXCEPTION}, 
        ],
        'epochsSearch': 10,
        'epochsBest': 20,
        'top': 5,
        'verbose': 1
    }
}

  
genderBestModels = researchStudy(genderStudy, dbStudyOutPath.format('GenderStudy/{}'))

Ablation for SOCOfingGender
Epoch 1/10
55/55 [==============================] - 15s 192ms/step - loss: 0.9191 - accuracy: 0.5213 - val_loss: 0.6811 - val_accuracy: 0.5444
Epoch 2/10
55/55 [==============================] - 5s 94ms/step - loss: 0.6998 - accuracy: 0.5557 - val_loss: 0.6704 - val_accuracy: 0.5889
Epoch 3/10
55/55 [==============================] - 5s 96ms/step - loss: 0.6688 - accuracy: 0.6103 - val_loss: 0.6610 - val_accuracy: 0.6333
Epoch 4/10
55/55 [==============================] - 5s 92ms/step - loss: 0.6550 - accuracy: 0.6253 - val_loss: 0.6607 - val_accuracy: 0.6194
Epoch 5/10
55/55 [==============================] - 5s 96ms/step - loss: 0.6319 - accuracy: 0.6552 - val_loss: 0.6671 - val_accuracy: 0.6167
Epoch 6/10
55/55 [==============================] - 5s 92ms/step - loss: 0.6308 - accuracy: 0.6690 - val_loss: 0.6431 - val_accuracy: 0.6389
Epoch 7/10
55/55 [==============================] - 5s 92ms/step - loss: 0.6080 - accuracy: 0.6839 - val_loss: 0.6455 - val_

INFO:tensorflow:Assets written to: ./out/GenderStudy/SOCOfingGender/FinalModels/Optimizer_Nadam/assets


INFO:tensorflow:Assets written to: ./out/GenderStudy/SOCOfingGender/FinalModels/Optimizer_Nadam/assets


Epoch 1/20
67/67 [==============================] - 9s 138ms/step - loss: 0.6061 - accuracy: 0.6867 - val_loss: 0.7235 - val_accuracy: 0.5750
Epoch 2/20
67/67 [==============================] - 7s 112ms/step - loss: 0.5839 - accuracy: 0.7124 - val_loss: 0.6465 - val_accuracy: 0.6583
Epoch 3/20
67/67 [==============================] - 8s 112ms/step - loss: 0.5677 - accuracy: 0.7219 - val_loss: 0.6386 - val_accuracy: 0.6278
Epoch 4/20
67/67 [==============================] - 7s 109ms/step - loss: 0.5623 - accuracy: 0.7238 - val_loss: 0.6491 - val_accuracy: 0.6306
Epoch 5/20
67/67 [==============================] - 8s 112ms/step - loss: 0.5553 - accuracy: 0.7310 - val_loss: 0.6945 - val_accuracy: 0.6000
Epoch 6/20
67/67 [==============================] - 7s 111ms/step - loss: 0.5346 - accuracy: 0.7495 - val_loss: 0.7048 - val_accuracy: 0.5917
Epoch 7/20
67/67 [==============================] - 8s 115ms/step - loss: 0.5362 - accuracy: 0.7405 - val_loss: 0.6686 - val_accuracy: 0.6278
Epoch 

INFO:tensorflow:Assets written to: ./out/GenderStudy/SOCOfingGender/FinalModels/LearningRate_1e-05/assets


...

INFO:tensorflow:Assets written to: ./out/GenderStudy/NISTSDB4Gender/FinalModels/ResNet-Classifier/assets


Epoch 1/20
107/107 [==============================] - 24s 225ms/step - loss: 0.0671 - accuracy: 0.9826 - val_loss: 0.4217 - val_accuracy: 0.8733
Epoch 2/20
107/107 [==============================] - 24s 228ms/step - loss: 0.0548 - accuracy: 0.9888 - val_loss: 0.4254 - val_accuracy: 0.8667
Epoch 3/20
107/107 [==============================] - 28s 267ms/step - loss: 0.0539 - accuracy: 0.9924 - val_loss: 0.4211 - val_accuracy: 0.8783
Epoch 4/20
107/107 [==============================] - 30s 284ms/step - loss: 0.0587 - accuracy: 0.9835 - val_loss: 0.4386 - val_accuracy: 0.8733
Epoch 5/20
107/107 [==============================] - 30s 280ms/step - loss: 0.0519 - accuracy: 0.9912 - val_loss: 0.4323 - val_accuracy: 0.8867
Epoch 6/20
107/107 [==============================] - 28s 263ms/step - loss: 0.0555 - accuracy: 0.9874 - val_loss: 0.4270 - val_accuracy: 0.8817
Epoch 7/20
107/107 [==============================] - 28s 263ms/step - loss: 0.0532 - accuracy: 0.9871 - val_loss: 0.4338 - val_ac

INFO:tensorflow:Assets written to: ./out/GenderStudy/NISTSDB4Gender/FinalModels/LossFunction_binary_crossentropy/assets


Epoch 1/20
107/107 [==============================] - 24s 225ms/step - loss: 0.0328 - accuracy: 0.9956 - val_loss: 0.4881 - val_accuracy: 0.8933
Epoch 2/20
107/107 [==============================] - 24s 225ms/step - loss: 0.0347 - accuracy: 0.9938 - val_loss: 0.5281 - val_accuracy: 0.8667
Epoch 3/20
107/107 [==============================] - 28s 258ms/step - loss: 0.0285 - accuracy: 0.9974 - val_loss: 0.4973 - val_accuracy: 0.8917
Epoch 4/20
107/107 [==============================] - 30s 285ms/step - loss: 0.0247 - accuracy: 0.9991 - val_loss: 0.5106 - val_accuracy: 0.8983
Epoch 5/20
107/107 [==============================] - 30s 283ms/step - loss: 0.0368 - accuracy: 0.9929 - val_loss: 0.5082 - val_accuracy: 0.8933
Epoch 6/20
107/107 [==============================] - 28s 263ms/step - loss: 0.0386 - accuracy: 0.9915 - val_loss: 0.5682 - val_accuracy: 0.8533
Epoch 7/20
107/107 [==============================] - 28s 261ms/step - loss: 0.0292 - accuracy: 0.9953 - val_loss: 0.4970 - val_ac

INFO:tensorflow:Assets written to: ./out/GenderStudy/NISTSDB4Gender/FinalModels/LearningRate_0.001/assets


#### **2. Finger Name** 
##### **Datasets:**
* **SOCOFing**
* **NIST Special Database 4**
* **NIST Special Database 300a Roll**
* **NIST Special Database 300b - All Scanners**

##### **Hyperparameters**
* Optimizers: *Adam, Nadam, RMSprop*
* Loss Functions: *categorical-crossentropy, mean-squared-error, categorical-hinge*
* Learning Rates: *0.1, 0.01, 0.001, 0.0001, 0.00001*
* Backbones: *MobileNetV2, ResNet50, EfficientNetB2, InceptionV3, Xception*

In [ ]:

fingerNameStudy = {
    'datasets' : [
        ds.SOCOFingFingers(**datasetsConfig), 
        ds.NISTSDB4Fingers(**datasetsConfig), 
        ds.NISTSDB300aFingers(**datasetsConfig), 
        ds.NISTSDB302bFingers(**datasetsConfig)
    ][:2],
    'studyHyperParameters' : {
        'defaultBackbone': {'backbone' : bb.MobileNetV2, 'weights' : weightsMNV2},
        'defaultClassifier': cf.DefaultClassifier,
        'defaultHypers': {
            "optimizer": Adam,
            "learningRate": 0.001,
            "loss": 'categorical_crossentropy',
            "metrics": 'accuracy'
        },
        'losses': ['mean_squared_error', 'categorical_hinge'][:1],
        'learningRates': [0.1, 0.01, 0.0001, 0.00001][:1],
        'optimizers': [Nadam, RMSprop][:1],
        'classifiers': [cf.AlexNetClassifier, cf.MobileNetClassifier, cf.ResNetClassifier][:1],
        'backbonesForSearch': [
            {'backbone' : bb.ResNet50, 'weights' : weightsRN50},
            {'backbone' : bb.EfficientNetB2, 'weights' : weightsENB2},
            {'backbone' : bb.InceptionV3, 'weights' : weightsINCEPTIONV3},
            {'backbone' : bb.Xception, 'weights' : weightsXCEPTION}, 
        ][:1],
        'epochsSearch': 2,
        'epochsBest': 2,
        'top': 2,
        'verbose': 1
    }
}

  
fingersBestModels = researchStudy(fingerNameStudy, dbStudyOutPath.format('FingerName/{}'))

In [14]:

fingersBestModels['NISTSDB4Fingers']
for modelName, model in fingersBestModels['NISTSDB4Fingers'].items():
    print(model['model'])

#### **3. Fingerprint type** 
##### **Datasets:**
* **NIST Special Database 4**

##### **Hyperparameters**
* Optimizers: *Adam, Nadam, RMSprop*
* Loss Functions: *categorical-crossentropy, mean-squared-error, categorical-hinge*
* Learning Rates: *0.1, 0.01, 0.001, 0.0001, 0.00001*
* Backbones: *MobileNetV2, ResNet50, EfficientNetB2, InceptionV3, Xception*

In [18]:

fingerprintTypeNameStudy = {
    'datasets' : [
        ds.NISTSDB4Type(**datasetsConfig), 
    ],
    'studyHyperParameters' : {
        'defaultBackbone': {'backbone' : bb.MobileNetV2, 'weights' : weightsMNV2},
        'defaultClassifier': cf.DefaultClassifier,
        'defaultHypers': {
            "optimizer": Adam,
            "learningRate": 0.001,
            "loss": 'categorical_crossentropy',
            "metrics": 'accuracy'
        },
        'losses': ['mean_squared_error', 'categorical_hinge'],
        'learningRates': [0.1, 0.01, 0.0001, 0.00001],
        'optimizers': [Nadam, RMSprop],
        'classifiers': [cf.AlexNetClassifier, cf.MobileNetClassifier, cf.ResNetClassifier],
        'backbonesForSearch': [
            {'backbone' : bb.ResNet50, 'weights' : weightsRN50},
            {'backbone' : bb.EfficientNetB2, 'weights' : weightsENB2},
            {'backbone' : bb.InceptionV3, 'weights' : weightsINCEPTIONV3},
            {'backbone' : bb.Xception, 'weights' : weightsXCEPTION}, 
        ],
        'epochsSearch': 10,
        'epochsBest': 20,
        'top': 5,
        'verbose': 1
    }
}


  
fingerprintTypeBestModels = researchStudy(fingerprintTypeNameStudy, dbStudyOutPath.format('FingerprintType/{}'))

Ablation for NISTSDB4Type
Epoch 1/10
88/88 [==============================] - 26s 220ms/step - loss: 1.9764 - accuracy: 0.2475 - val_loss: 1.4734 - val_accuracy: 0.3750
Epoch 2/10
88/88 [==============================] - 10s 119ms/step - loss: 1.4081 - accuracy: 0.3646 - val_loss: 1.1666 - val_accuracy: 0.4817
Epoch 3/10
88/88 [==============================] - 10s 110ms/step - loss: 1.2586 - accuracy: 0.4332 - val_loss: 1.1291 - val_accuracy: 0.5183
Epoch 4/10
88/88 [==============================] - 10s 110ms/step - loss: 1.1977 - accuracy: 0.4561 - val_loss: 1.0596 - val_accuracy: 0.5267
Epoch 5/10
88/88 [==============================] - 9s 106ms/step - loss: 1.1401 - accuracy: 0.5004 - val_loss: 1.0526 - val_accuracy: 0.5467
Epoch 6/10
88/88 [==============================] - 9s 106ms/step - loss: 1.1302 - accuracy: 0.5004 - val_loss: 1.1621 - val_accuracy: 0.4467
Epoch 7/10
88/88 [==============================] - 10s 119ms/step - loss: 1.0807 - accuracy: 0.5132 - val_loss: 1.080

INFO:tensorflow:Assets written to: ./out/FingerprintType/NISTSDB4Type/FinalModels/LearningRate_0.0001/assets


Epoch 1/20
107/107 [==============================] - 31s 289ms/step - loss: 0.5439 - accuracy: 0.8062 - val_loss: 1.2835 - val_accuracy: 0.5733
Epoch 2/20
107/107 [==============================] - 29s 273ms/step - loss: 0.5263 - accuracy: 0.8041 - val_loss: 0.8548 - val_accuracy: 0.6783
Epoch 3/20
107/107 [==============================] - 33s 308ms/step - loss: 0.4540 - accuracy: 0.8303 - val_loss: 0.9810 - val_accuracy: 0.6433
Epoch 4/20
107/107 [==============================] - 37s 347ms/step - loss: 0.3789 - accuracy: 0.8618 - val_loss: 1.1083 - val_accuracy: 0.6367
Epoch 5/20
107/107 [==============================] - 37s 348ms/step - loss: 0.3467 - accuracy: 0.8644 - val_loss: 0.9706 - val_accuracy: 0.6667
Epoch 6/20
107/107 [==============================] - 34s 317ms/step - loss: 0.2966 - accuracy: 0.8918 - val_loss: 0.9110 - val_accuracy: 0.6617
Epoch 7/20
107/107 [==============================] - 35s 323ms/step - loss: 0.2598 - accuracy: 0.9103 - val_loss: 1.1627 - val_ac

INFO:tensorflow:Assets written to: ./out/FingerprintType/NISTSDB4Type/FinalModels/Optimizer_Nadam/assets


Epoch 1/20
107/107 [==============================] - 26s 243ms/step - loss: 0.1205 - accuracy: 0.9556 - val_loss: 1.1615 - val_accuracy: 0.6783
Epoch 2/20
107/107 [==============================] - 27s 254ms/step - loss: 0.0972 - accuracy: 0.9674 - val_loss: 1.1327 - val_accuracy: 0.6850
Epoch 3/20
107/107 [==============================] - 33s 307ms/step - loss: 0.1091 - accuracy: 0.9579 - val_loss: 1.1487 - val_accuracy: 0.6933
Epoch 4/20
107/107 [==============================] - 34s 317ms/step - loss: 0.1074 - accuracy: 0.9615 - val_loss: 1.2960 - val_accuracy: 0.6633
Epoch 5/20
107/107 [==============================] - 34s 318ms/step - loss: 0.1050 - accuracy: 0.9635 - val_loss: 1.1320 - val_accuracy: 0.6983
Epoch 6/20
107/107 [==============================] - 34s 316ms/step - loss: 0.1131 - accuracy: 0.9609 - val_loss: 1.2369 - val_accuracy: 0.6883
Epoch 7/20
107/107 [==============================] - 33s 308ms/step - loss: 0.1142 - accuracy: 0.9579 - val_loss: 1.3124 - val_ac

INFO:tensorflow:Assets written to: ./out/FingerprintType/NISTSDB4Type/FinalModels/Optimizer_Adam/assets


Epoch 1/20
107/107 [==============================] - 33s 300ms/step - loss: 0.9086 - accuracy: 0.5985 - val_loss: 0.9086 - val_accuracy: 0.6100
Epoch 2/20
107/107 [==============================] - 27s 249ms/step - loss: 0.9281 - accuracy: 0.5771 - val_loss: 0.9704 - val_accuracy: 0.6017
Epoch 3/20
107/107 [==============================] - 30s 279ms/step - loss: 0.8699 - accuracy: 0.5950 - val_loss: 0.9133 - val_accuracy: 0.6250
Epoch 4/20
107/107 [==============================] - 37s 342ms/step - loss: 0.8776 - accuracy: 0.5971 - val_loss: 0.8874 - val_accuracy: 0.6217
Epoch 5/20
107/107 [==============================] - 36s 336ms/step - loss: 0.8221 - accuracy: 0.6191 - val_loss: 0.8863 - val_accuracy: 0.6200
Epoch 6/20
107/107 [==============================] - 31s 294ms/step - loss: 0.8263 - accuracy: 0.6303 - val_loss: 0.9275 - val_accuracy: 0.6017
Epoch 7/20
107/107 [==============================] - 33s 305ms/step - loss: 0.7909 - accuracy: 0.6512 - val_loss: 0.9777 - val_ac

INFO:tensorflow:Assets written to: ./out/FingerprintType/NISTSDB4Type/FinalModels/AlexNet-Classifier/assets


Epoch 1/20
107/107 [==============================] - 26s 240ms/step - loss: 0.7014 - accuracy: 0.6865 - val_loss: 0.8861 - val_accuracy: 0.6250
Epoch 2/20
107/107 [==============================] - 28s 262ms/step - loss: 0.6773 - accuracy: 0.6926 - val_loss: 0.8850 - val_accuracy: 0.6050
Epoch 3/20
107/107 [==============================] - 35s 331ms/step - loss: 0.6716 - accuracy: 0.7026 - val_loss: 0.9001 - val_accuracy: 0.6183
Epoch 4/20
107/107 [==============================] - 36s 336ms/step - loss: 0.6888 - accuracy: 0.6947 - val_loss: 0.8947 - val_accuracy: 0.6317
Epoch 5/20
107/107 [==============================] - 32s 301ms/step - loss: 0.6747 - accuracy: 0.6944 - val_loss: 0.9153 - val_accuracy: 0.5917
Epoch 6/20
107/107 [==============================] - 32s 295ms/step - loss: 0.6225 - accuracy: 0.7215 - val_loss: 0.9056 - val_accuracy: 0.5933
Epoch 7/20
107/107 [==============================] - 34s 318ms/step - loss: 0.6575 - accuracy: 0.7112 - val_loss: 0.9537 - val_ac

INFO:tensorflow:Assets written to: ./out/FingerprintType/NISTSDB4Type/FinalModels/LossFunction_categorical_crossentropy/assets


#### **4. Same Person** 
##### **Datasets:**
* **NIST Special Database 4**

##### **Hyperparameters**
* Optimizers: *Adam, Nadam, RMSprop*
* Loss Functions: *binary-crossentropy, mean-squared-error, hinge*
* Learning Rates: *0.1, 0.01, 0.001, 0.0001, 0.00001*
* Backbones: *MobileNetV2, ResNet50, EfficientNetB2, InceptionV3, Xception*

In [ ]:

samePersonStudy = {
    'datasets' : [
        # ds.SOCOFingSamePerson(**datasetsConfig), 
        ds.NISTSDB300aSamePerson(**datasetsConfig), 
        # ds.NISTSDB302bSamePerson(**datasetsConfig), 
    ],
    'studyHyperParameters' : {
        'defaultBackbone': {'backbone' : bb.MobileNetV2, 'weights' : weightsMNV2},
        'defaultClassifier': cf.DefaultClassifier,
        'defaultHypers': {
            "optimizer": Adam,
            "learningRate": 0.001,
            "loss": 'binary_crossentropy',
            "metrics": 'accuracy'
        },
        'losses': ['mean_squared_error', 'hinge'],
        'learningRates': [0.1, 0.01, 0.0001, 0.00001],
        'optimizers': [Nadam, RMSprop],
        'classifiers': [cf.AlexNetClassifier, cf.MobileNetClassifier, cf.ResNetClassifier],
        'backbonesForSearch': [
            {'backbone' : bb.ResNet50, 'weights' : weightsRN50},
            {'backbone' : bb.EfficientNetB2, 'weights' : weightsENB2},
            {'backbone' : bb.InceptionV3, 'weights' : weightsINCEPTIONV3},
            {'backbone' : bb.Xception, 'weights' : weightsXCEPTION}, 
        ],
        'epochsSearch': 10,
        'epochsBest': 20,
        'top': 5,
        'verbose': 1
    }
}


  
samePersonBestModels = researchStudy(samePersonStudy, dbStudyOutPath.format('SamePerson/{}'))

In [ ]:
def evaluteStudyOnOthersDatasets(bestModels, datasets, epochsFinal, verbose=1, path=None):
    
    evaluations = {}
    for datasetName, bestDatasetModels in bestModels.items():
        chainedTrain = None
        chainedTest = None
        for dset in datasets:
            dset.split = [1]
            [toConcat] = dset.create()
            if(dset.name != datasetName):
                if chainedTrain == None:
                    chainedTrain = toConcat
                else:
                    chainedTrain = chainedTrain.concatenate(toConcat)
            else:
                if chainedTest == None:
                    chainedTest = toConcat
                else:
                    chainedTest.concatenate(toConcat)
        evaluations[datasetName] = {}
        for modelName, model in bestDatasetModels.items():
            evaluation = model['model'].getModel().evaluate(chainedTrain)
            evaluations[datasetName][modelName] = {
                'evaluation': evaluation,
                'train' : chainedTrain, 
                'test': chainedTest
            }
    return evaluations    

    # finalModelConfiguration = bestModels[bestModelName][1]
    # finalModel = FPML(**finalModelConfiguration["architecture"]).create(**finalModelConfiguration["hyperparameters"])
    # finalModelHistory = finalModel.fit(finalDs['train'], validation_data=finalDs['test'], epochs=epochsFinal, verbose=verbose)
    
    # utils.datasetAnalysisAndDisplay([finalDs['train'], finalDs['test']], datasets[0], path)
    
    # toSave = {'Final Model':{'history' : finalModelHistory, 'config' : finalModelConfiguration}}
    
    # finalModel.save(path+'FinalModel-'+bestModelName)
    # utils.displayStudiesProgress(toSave, path, 'Final Model')
    # utils.saveStudyHistory({'Final Model': finalModelConfiguration}, toSave, path+'finalModel.csv')
    # utils.displayConfusion(finalDs['test'], finalModel, path)


evaluations = evaluteStudyOnOthersDatasets(fingersBestModels, fingerNameStudy['datasets'], 2, 1)

In [66]:
import csv
from matplotlib import pyplot as plt

# Path('./final_result_full_history/').mkdir(parents=True, exist_ok=True)
# for model in allBestModels:
def saveTopModelsFullHistory(bestModels, path):
    
    for key, val in bestModels.items():
        datasetPath = path+key+'/'
        Path(path+key).mkdir(parents=True, exist_ok=True)
        f = open(datasetPath+'full_history.csv', 'w')
        writer = csv.writer(f)
        for key2, val2 in val.items():
            acc = ['Accuracy']
            val_acc = ['Val Accuracy']
            loss = ['Loss']
            val_loss = ['Validation Loss']
            writer.writerow([key2])
            for history in val2['model'].history:
                acc = [*acc, *history.history['accuracy']]
                val_acc = [*val_acc, *history.history['val_accuracy']]
                loss = [*loss, *history.history['loss']]
                val_loss = [*val_loss, *history.history['val_loss']]
            writer.writerow(acc)
            writer.writerow(val_acc)
            writer.writerow(loss)
            writer.writerow(val_loss)
            writer.writerow(['', ''])
            plt.figure(figsize=(10, 10))
            fig, ax1 = plt.subplots()

            ax2 = ax1.twinx()
            ax1.plot(np.asarray(acc[1:], float), 'o-', color="black" )
            ax1.plot(np.asarray(val_acc[1:], float), 'o-', color="gray" )
            ax2.plot(np.asarray(loss[1:], float), '-', color="red" )
            ax2.plot(np.asarray(val_loss[1:], float), '-', color="green" )

            ax1.legend(['train-acc', 'test-acc'])
            ax2.legend(['train-loss', 'test-loss'])
            ax1.set_xlabel('epochs')
            ax1.set_ylabel('Accuracy', color='g')
            ax2.set_ylabel('Loss', color='b')
            plt.grid()
            plt.savefig(datasetPath+key2+'.png', bbox_inches='tight')
            plt.close()

        f.close()

saveTopModelsFullHistory(fingerprintTypeBestModels, path='./zz_final_result_full_history/fingerprintType/')
saveTopModelsFullHistory(fingersBestModels, path='./zz_final_result_full_history/fingerName/')
saveTopModelsFullHistory(genderBestModels, path='./zz_final_result_full_history/gender/')





/var/folders/21/cz8tyh557md2q7c2q81f16b80000gn/T/ipykernel_9485/2578401039.py:30: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax1 = plt.subplots()


<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

...

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>